In [17]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = data.data
y = data.target

print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: (569, 30)
Target shape: (569,)


In [18]:
print("Attribute names:", data.feature_names)

Attribute names: ['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [19]:
df = pd.DataFrame(X, columns=data.feature_names)
print(df.head())

   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst radius  worst texture  worst perimeter  \
0           

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

In [21]:
import numpy as np
print("Missing values per feature:", np.isnan(X).sum(axis=0))

Missing values per feature: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [22]:
### This would've been used if the data had missing values
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='mean')
# X_imputed = imputer.fit_transform(X)

In [23]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
### Encoding is not necessary for this dataset as it contains only numerical features.

# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()
# y_encoded = le.fit_transform(y)

# from sklearn.preprocessing import OneHotEncoder

# encoder = OneHotEncoder(sparse=False)
# X_encoded = encoder.fit_transform(X_categorical)

In [ ]:
# Feature selection

selected_features = [
    'mean radius',
    'mean texture',
    'mean perimeter',
    'mean area',
    'mean smoothness'
]

df_selected = df[selected_features]
print(df_selected.head())

   mean radius  mean texture  mean perimeter  mean area  mean smoothness
0        17.99         10.38          122.80     1001.0          0.11840
1        20.57         17.77          132.90     1326.0          0.08474
2        19.69         21.25          130.00     1203.0          0.10960
3        11.42         20.38           77.58      386.1          0.14250
4        20.29         14.34          135.10     1297.0          0.10030


In [28]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, f1_score

# Use selected features for training
X_train_selected = X_train[:, [df.columns.get_loc(f) for f in [
    'mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness'
]]]
X_test_selected = X_test[:, [df.columns.get_loc(f) for f in [
    'mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness'
]]]

# Create pipelines for each classifier
logreg = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression())
])

svm = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SVC(probability=False))
])

knn = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsClassifier())
])

dtree = Pipeline([
    ('clf', DecisionTreeClassifier())
])

nb = Pipeline([
    ('clf', GaussianNB())
])

# Create VotingClassifier with hard voting
voting_clf = VotingClassifier(
    estimators=[
        ('lr', logreg),
        ('svm', svm),
        ('knn', knn),
        ('dt', dtree),
        ('nb', nb)
    ],
    voting='hard'
)

# Cross-validation on training set
cv_scores = cross_val_score(voting_clf, X_train_selected, y_train, cv=5)
print(f"Voting Classifier Cross-Validation Accuracy (mean): {cv_scores.mean():.4f}")

# Fit and evaluate on test set
voting_clf.fit(X_train_selected, y_train)
y_pred = voting_clf.predict(X_test_selected)

test_accuracy = voting_clf.score(X_test_selected, y_test)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)

print(f"Voting Classifier Test Accuracy: {test_accuracy:.4f}")
print(f"Voting Classifier Test Recall: {test_recall:.4f}")
print(f"Voting Classifier Test F1 Score: {test_f1:.4f}")

Voting Classifier Cross-Validation Accuracy (mean): 0.9120
Voting Classifier Test Accuracy: 0.9532
Voting Classifier Test Recall: 0.9630
Voting Classifier Test F1 Score: 0.9630
